<a href="https://colab.research.google.com/github/ThapasPPramod/100DaysOfCode/blob/main/Quiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install prettytable

In [2]:
import requests, json, random, html
from bs4 import BeautifulSoup
from IPython.display import clear_output
from prettytable import PrettyTable

In [3]:
class User():
  def __init__(self):
    self.i = 1
    self.score = 0
    self.correct = 2
    self.incorrect = -0.25

In [7]:
class Data(User):
  def __init__(self):
    super().__init__()
    self.getCategories()
  def checkBool(self, choice:str, ans):
    ans = html.unescape(ans)
    choice = choice.title()
    if len(choice) == 1:
      if choice == 'T':
        return  ans == 'True'
      elif choice == 'F':
        return ans == 'False'
    else:
      return ans == choice
  def checkMultiple(self, list_,choice, ans):
    choice = int(choice)
    if choice > 0 and choice < 5:
      return list_[choice-1] == ans
    return False
  def process(self, q):
    i, tf = self.i, ''
    self.i += 1
    try:
      question = html.unescape(q['question'])
      m = False
      print(f"Q {i}). {question}\n")
      if q['type'] == 'multiple':
        m = True
        options = q['incorrect_answers']
        options.append(q['correct_answer'])
        trackoptions = dict()
        for ind in range(4):
          x = random.choice(options)
          options.remove(x)
          x = html.unescape(x)
          trackoptions[ind] = x
          print(f'{ind+1} : {x}')
      else:
        tf = ' True or False '

      choice = input(f'Enter your guess{tf}: ').strip()

      if (m and self.checkMultiple(trackoptions,choice, html.unescape(q['correct_answer']))) or (not m and self.checkBool(choice, q['correct_answer'])):
        self.score += self.correct
        print(f'You guessed the right answer! Your score : {self.score}')
        return True
      self.score += self.incorrect
      print(f"The correct answer was {html.unescape(q['correct_answer'])}! Your score : {self.score}")
      return True
    except Exception as e:
      print('Error: ', e)
      return False

  def getCategories(self):
    try:
      response = requests.get("https://opentdb.com/api_config.php")
      if response.status_code == 200:
        data = BeautifulSoup(response.content,'html.parser')
        forms = data.find_all('form')
        selects = forms[0].find_all('select')
        category = [select for select in selects if select.get('name') == 'trivia_category']
        options = category[0].find_all('option')
        self.categories = list()
        self.tableOfCat = PrettyTable(field_names=['Index','Categories'])
        i = 0
        for option in options:
          cat = option.text.strip()
          self.categories.append(option.get('value'))
          self.tableOfCat.add_row([i, cat])
          i += 1
    except Exception as e:
      print('Can\'t access the website', e)

  def findCategory(self):
    cat = input(f'Enter category of questions to be asked from the table\n{self.tableOfCat}\n : ')
    try:
      return self.categories[int(cat.strip())]
    except Exception as e:
      print('Continuing with random set of questions!')
      return self.categories[0]

  def getFromUrl(self,url):
    try:
      response = requests.get(url)
      if response.status_code == 200:
        # print(response)
        json_content = response.json()
        # print(json_content)
        return json_content['results']
    except Exception as e:
      print('Error : ', e)
      return None

  def getData(self):
    try:
      nq = input('Enter no. of questions(1-50) : ').strip()
      if not nq.isdigit() or int(nq) > 50 or int(nq) < 1:
        nq = '10'
      diff, qtype = 'a', 'any'
      cat = self.findCategory()
      diff = input('Enter difficulty level(Easy(e), Medium(m) or Hard(h) or Any(a))? : ').strip().lower()
      difficulty = {'a':'a' ,'e':'easy', 'm':'medium', 'h':'hard'}
      qtype = input('Enter type of questions to be asked(Boolean (b) or Multiple choice (m) or Random (r)) : ').strip().lower()
      qtype = 'boolean' if qtype == 'b' else ('multiple' if qtype == 'm' else 'any')
      url = f'https://opentdb.com/api.php?amount={nq}'
      url = url + f"&category={cat}" if cat != 'any' else url
      url = url + f"&type={qtype}" if qtype != 'any' else url
      url = url + f"&difficulty={difficulty[diff]}" if diff != 'a' else url
      # print(url)
      return self.getFromUrl(url)
    except Exception as e:
      print('Error : ',e)
      return None

In [5]:
def main():
  retry = False
  data = Data()
  while True:
    if retry:
      choice = input(f'Your score was {data.score}! Quit?(y/n) : ').strip()
      if choice in ('y', 'Y'):
        break
    question_bank = data.getData()

    if question_bank and (input('Begin quiz?(y/n) : ') in ('y', 'Y')):
      while len(question_bank) > 0:
        clear_output(wait=True)
        question = random.choice(question_bank)
        if data.process(question):
            question_bank.remove(question)
        choice = input('Continue? (y/n) : ').strip()
        if choice in ('n', 'N'):
          break
    retry = True

In [8]:
main()

Q 5). The protagonist's names in 'Who's Afraid of Virginia Woolf', George and Martha, were derived from George Washington and his wife.

Enter your guess True or False : t
You guessed the right answer! Your score : 1.0
Continue? (y/n) : y
Your score was 1.0! Quit?(y/n) : y
